# Project 4: Stability of three algorithms for solving Least Squares

In this project, we will see how stable three different algorithms for solving the same least square problem are, with respect to round off error. The theory to explain why some are more stable than another is quite involved and probably takes half of a semester to go through. If you are interested, the details are in the recommended text "Numerical Linear Algebra" by Trefethen and Bau. 

**Your name here:**    <i>Kevin Wong</i>

## Setting up a Least Squares problem

In [ ]:
import numpy as np

m = 100
n = 15
t = np.arange(0,1+1.0/(m-1),1.0/(m-1)) # Set t to a discretization of [0,1]. len(t)=m
A = np.array([t**i for i in range(n)]).T # Construct a submatrix of a Vandermonde matrix
#size of A is m by n

#truex is the real least square solution
truex = np.array([-0.76913135,  0.46167844,  0.10294497, -0.55750683,  1.37792289,  1.1454379,
   0.52179532, -2.59420408,  0.0355606,   1.67058624,  0.1212572,  -1.14884385,
  -0.78537181, -1.18751783, 1])

#Construct the right-hand side vector b so that least square solution of Ax=b is truex
U,S,V = np.linalg.svd(A)
AC = U[:,15:]
b = A.dot(truex).reshape(100,1) + np.dot(AC, np.ones([85,1]))

In [2]:
# b is constructed from the least-square solution vector transformed by matrix A, 
# shifted by the combined values of the left null-space of A (the null-space of A transpose).  
# My theory is that this shift ensures that A.T x A is invertible and has a unique solution,
# for every vector it acts upon, which is necessary in finding the least-square solution.  

### Below you will need to use three different algorithms that we talked about for solving LS. Use functions like solve, qr, svd... from the linalg package. 

In order to compare the solution with truex, we will just compare the last entry, which should be 1.

## 1. Via the normal equation

(1) Directly use solve. No need to do cholesky (the solve function is doing that). 

Call the solution xn. You need to print running time (you can run a couple times to get a stable time), print the last entry difference: abs(xn[14]-1)

In [3]:
import time

running_times = []
for trial in range(1000):
    start = time.time()
    xn = np.linalg.solve(np.dot(A.T,A), np.dot(A.T,b))
    end = time.time()
    running_times.append(end-start)

print "Average running time of 1000 trials: " + str(np.mean(running_times)) + " seconds"

Average running time of 1000 trials: 4.56967353821e-05 seconds


In [4]:
abs(xn[14]-1)

array([ 1.6600671])

In [5]:
xn - truex.reshape(15,1)

array([[ -1.30417299e-07],
       [  3.81524305e-05],
       [ -1.86031907e-03],
       [  3.65769355e-02],
       [ -3.77956086e-01],
       [  2.32426530e+00],
       [ -9.06958888e+00],
       [  2.30675918e+01],
       [ -3.78799924e+01],
       [  3.71821886e+01],
       [ -1.47123808e+01],
       [ -1.03860778e+01],
       [  1.67838246e+01],
       [ -8.62669602e+00],
       [  1.66006710e+00]])

## 2. Via reduced QR

(2) Solve LS via reduced QR. 

Call the solution xq. You need to print running time, print the last entry difference: abs(xq[14]-1)

In [6]:
running_times = []
for trial in range(1000):
    start = time.time()
    Q, R = np.linalg.qr(A, mode='reduced')
    xq = np.linalg.solve(R, Q.T.dot(b))
    end = time.time()
    running_times.append(end-start)
    
print "Average running time of 1000 trials: " + str(np.mean(running_times)) + " seconds"

Average running time of 1000 trials: 0.000145689487457 seconds


In [7]:
abs(xq[14]-1)

array([  3.87883814e-09])

In [8]:
xq - truex.reshape(15,1)

array([[ -1.99840144e-15],
       [  1.57596158e-13],
       [ -4.90238405e-12],
       [  7.61167795e-11],
       [ -7.15220994e-10],
       [  4.44755965e-09],
       [ -1.91434886e-08],
       [  5.84316484e-08],
       [ -1.28020919e-07],
       [  2.01769530e-07],
       [ -2.26686306e-07],
       [  1.77117487e-07],
       [ -9.14664615e-08],
       [  2.80736383e-08],
       [ -3.87883814e-09]])

## 3. Via reduced SVD

(3) Solve LS via reduced SVD. 

Call the solution xs. You need to print running time, print the last entry difference: abs(xs[14]-1)

In [9]:
U,S,V = np.linalg.svd(A, full_matrices=False)
print V.shape
xs = np.linalg.solve(np.diag(S).dot(V), U.T.dot(b))

(15, 15)


In [10]:
running_times = []
for trial in range(1000):
    start = time.time()
    U,S,V = np.linalg.svd(A, full_matrices=False)
    V.shape
    xs = np.linalg.solve(np.diag(S).dot(V), U.T.dot(b))
    end = time.time()
    running_times.append(end-start)
    
print "Average running time of 1000 trials: " + str(np.mean(running_times)) + " seconds"

Average running time of 1000 trials: 0.000181740999222 seconds


In [11]:
abs(xs[14]-1)

array([  4.37934133e-10])

In [12]:
xs - truex.reshape(15,1)

array([[ -2.44249065e-15],
       [  1.24622535e-13],
       [ -3.87309629e-12],
       [  6.01325656e-11],
       [ -5.46113821e-10],
       [  3.17614224e-09],
       [ -1.24466701e-08],
       [  3.38289721e-08],
       [ -6.46778505e-08],
       [  8.71238028e-08],
       [ -8.16963643e-08],
       [  5.17354115e-08],
       [ -2.08270639e-08],
       [  4.71128625e-09],
       [ -4.37934133e-10]])

(4) Write a paragraph (in this cell) commenting on stability and running time of each one:

Finding the least squared solution of Ax=b is done using three different approaches.  The normal equation has the most error, but is the fastest by an order of magnitude.  Decomposing A into a Q and R matrix and solving QRx = QQ.Tb is a more balanced approach, with dramatic increases in accuracy but slowing the computation down by an order of magnitude (compared to the normal equation).  The last method involves finding the reduced singular value decomposition of A, and solving USV.T = UU.Tb.  This involves more computations than either of the preceeding two methods, and takes between 1 and 2 times as long as QR decomposition, however, the error is smallest and stability is the best with this method.  
